# Random sample values for same date sentinel-1 and sentinel-2 images

- Find the same date optical image from sentinel-2 for each senetinel-1 image

- Speckle filter for the sentinel-1 image

- For each sentinel-2 image, keep images with tile cloud coverage < 20%

- Perform random sample --> 1000 points for each lake

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import geemap
import ee
import rasterio
import contextlib
from p_tqdm import p_map

In [2]:
# earthaccess.login()

In [3]:
ee.Authenticate()

True

In [4]:
ee.Initialize()

# Import datasets

In [5]:
filterSizeMeters = 60 # speckle filter size, in meter
scale = None # export image resolution, in meter

In [6]:
def filterSpeckles(img):
    
    # Function provided by Abhishek Kumar
    vv = img.select("VV")
    # spf: speckle filtered
    vv_filtered = vv.focal_median(filterSizeMeters, 'circle','meters').rename("VV_spf")
    
    return img.addBands(vv_filtered)

In [8]:
# def download_s1s2_sameday_images(lake_id, # export 60 meter resolution
#                            ):
# hydrolakes
lake_id = 1440

hydrolakes = ee.FeatureCollection("projects/ee-lakeice/assets/HydroLAKES_polys_v10")        
aoi = hydrolakes.filterMetadata('Hylak_id', 'equals', lake_id).first()
roi = aoi.geometry()

# Load SENTINEL-1
s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filterDate("2014-01-01", "2023-12-31") \
        .filterBounds(roi)
# speckle filter
# add image date to the property
s1_with_dates = s1.map(lambda image: filterSpeckles(image).set('date', 
                                               ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss')))

In [11]:
# Landsat 8 & 9
ls8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
        .filterDate('2013-01-01', '2023-12-31') \
        .filterBounds(roi)
# add a property as date
ls8_with_dates = s2.map(lambda image: image.set('date',
                                                ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss')))

# get dates
s1_dates = s1_with_dates.aggregate_array("date").getInfo()
s2_dates = s2_with_dates.aggregate_array("date").getInfo()
s1_dates = pd.to_datetime(s1_dates)
s2_dates = pd.to_datetime(s2_dates)

# Find sentinel-2 image that was acquired < 12 hour to the sentinel-1 image
def lookfor_sen2(sen1_date, 
                 sen2_dates = s2_dates, # list of dates of avaialble sentinel-2 images
                 delta_hr = 12):
    # find all times that has less than 12 hour interval to the sentinel-2 image
    intervals = np.abs(sen2_dates - sen1_date)
    indexes = intervals < pd.Timedelta(f"{delta_hr}H")

    # get the dates
    match_sen2_date = sen2_dates[indexes]

    if len(match_sen2_date) == 0:
        return None
    else:
        return match_sen2_date

# create a match table between sentinel-1 dates and sentinel-2 dates
match_table = pd.DataFrame(s1_dates, columns = ["s1_dates"])
match_table["s2_dates"] = match_table["s1_dates"].apply(lookfor_sen2)
match_table = match_table.dropna().reset_index(drop = True).explode("s2_dates")

# Find Sentinel-1
s1_date_string = ee.List(list(match_table.s1_dates.apply(lambda date: date.strftime('%Y-%m-%d %X')).unique()))
s2_date_string = ee.List(list(match_table.s2_dates.apply(lambda date: date.strftime('%Y-%m-%d %X')).unique()))

# create image collections of the sentinel-1, sentinel-2 (SCL)
matched_s1_img_col = s1_with_dates.filter(ee.Filter.inList('date',s1_date_string)).select(["VV_spf", "VH", "angle"])
matched_s2_img_col = s2_with_dates.filter(ee.Filter.inList('date',s2_date_string)).select(["B4", "B3", "B2", "SCL"])    

# create points based on the bound of the lake shape file
# buffer 10km
region = roi.buffer(10000).bounds()

# create ramdom points inside the lake bound
points = ee.FeatureCollection.randomPoints(
    region=region, points=1000, seed=0, maxError=1
)

out_file_ice = "{lake_id}_training_data.csv"

geemap.zonal_statistics(ice,
                        lakes,
                        out_file_ice, 
                        statistics_type='SUM', # no matter sum or other, same for points
                        scale=500)

In [12]:
test_points

VH     VV_spf      angle
0   -38.378636 -17.601433  43.996433
1   -31.934861 -19.539095  42.523235
2   -27.884636 -17.663347  43.226341
3   -30.898668 -16.056941  44.258106
4   -28.291745 -15.418497  44.288582
..         ...        ...        ...
414 -23.037096 -13.416638  43.314133
415 -23.382047 -14.449697  43.031887
416 -35.287738 -15.281299  42.785072
417 -20.448513 -11.155962  44.297607
418 -23.688283  -9.490974  42.791264

[419 rows x 3 columns]

In [9]:
lake_ids = [9086, 8000]

In [10]:
for lake_id in lake_ids:
    download_s1s2_sameday_images(lake_id)

Total number of images: 21



...0822T235738_20190822T235753_028689_033F46_24D9.tif: | | 0.00/14.8M (raw) [  0

...0218T235736_20200218T235752_031314_039A54_3248.tif: | | 0.00/14.8M (raw) [  0

...0418T235737_20200418T235753_032189_03B8E9_98D7.tif: | | 0.00/14.8M (raw) [  0

...0617T235740_20200617T235756_033064_03D487_EBD7.tif: | | 0.00/14.8M (raw) [  0

...

20240105T182741_20240105T182741_T12UVA.tif: | | 0.00/4.55M (raw) [  0.0%] in 00:

20240509T181919_20240509T182535_T12UVA.tif: | | 0.00/4.55M (raw) [  0.0%] in 00:

20240509T181919_20240509T182535_T12UWA.tif: | | 0.00/4.55M (raw) [  0.0%] in 00:

20240621T182919_20240621T183712_T12UVA.tif: | | 0.00/4.55M (raw) [  0.0%] in 00:

20240626T182921_20240626T183405_T12UWA.tif: | | 0.00/4.55M (raw) [  0.0%] in 00: